## Deliverable 2. Create a Customer Travel Destinations Map.

In [44]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [45]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lattes,FR,43.57,3.90,69.80,64,79,3.36,broken clouds
1,1,Turtas,RU,58.94,69.13,56.16,87,1,3.09,clear sky
2,2,Nekrasovskoye,RU,57.68,40.37,41.00,100,0,3.96,clear sky
3,3,Rikitea,PF,-23.12,-134.97,71.85,81,38,8.19,scattered clouds
4,4,Purranque,CL,-40.92,-73.17,37.40,100,66,2.44,broken clouds


In [46]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [47]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [48]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                232
City                   232
Country                232
Lat                    232
Lng                    232
Max Temp               232
Humidity               232
Cloudiness             232
Wind Speed             232
Current Description    232
dtype: int64

In [49]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [50]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Kavieng,PG,83.89,light rain,-2.57,150.80,
13,Butaritari,KI,83.91,clear sky,3.07,172.79,
16,Marawi,PH,82.00,broken clouds,8.00,124.29,
17,Cabo San Lucas,MX,86.00,overcast clouds,22.89,-109.91,
23,Zhuhai,CN,80.60,light intensity shower rain,22.28,113.57,
24,Mulatupo,PA,78.24,overcast clouds,8.95,-77.77,
25,Havelock,US,81.00,clear sky,34.88,-76.90,
31,Avarua,CK,75.20,light rain,-21.21,-159.78,
33,Atuona,PF,79.11,clear sky,-9.80,-139.03,
37,Kahramanmaras,TR,75.20,clear sky,38.00,37.00,


In [51]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found... skipping.')
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [54]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df['Hotel Name']  == ''].index)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Kavieng,PG,83.89,light rain,-2.57,150.80,Nusa Island Retreat
13,Butaritari,KI,83.91,clear sky,3.07,172.79,Isles Sunset Lodge
16,Marawi,PH,82.00,broken clouds,8.00,124.29,Salihmaruhom
17,Cabo San Lucas,MX,86.00,overcast clouds,22.89,-109.91,Hotel Tesoro Los Cabos
23,Zhuhai,CN,80.60,light intensity shower rain,22.28,113.57,珠海粤财假日酒店
25,Havelock,US,81.00,clear sky,34.88,-76.90,Sherwood Motel
31,Avarua,CK,75.20,light rain,-21.21,-159.78,Paradise Inn
33,Atuona,PF,79.11,clear sky,-9.80,-139.03,Villa Enata
39,Vanimo,PG,86.88,light rain,-2.67,141.30,Vahmoneh Lodge Vanimo
51,Kahului,US,89.60,few clouds,20.89,-156.47,Maui Seaside Hotel


In [55]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [73]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
#<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [74]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))